In [1]:
# !pip install cartopy
# import cartopy

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import glob
from PIL import Image

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# "/content/drive/MyDrive/DV_Dataset/*"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def get_mins(filepath:str):
    '''
    Processes a ascii file with gridded data to obtain the required value in the form of [latitude, longitude, value.
        Args:
            filepath: str
        Returns:
            metadata: Dict
            values: List[List[float]]
    '''
    metadata = {}
    longitudes = []
    values = []
    sizes = []
    i = 0
    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if i == 0: metadata["varname"] = line.split(":")[1].strip()
            if i == 1: metadata["filename"] = line.split(":")[1].strip()
            if i == 2: pass
            if i == 3: metadata["badflag"] = line.split(":")[1].strip()
            if i == 4: metadata["subset"] = line.split(":")[1].strip()
            if i == 5: metadata["time"] = line.split(":")[1].strip()
            if i == 6:
                for idx, l in enumerate(line.split('\t')):
                    try:
                        if l.strip()=='** line too long **':
                            continue
                        else:
                            longitudes.append(float(l.strip()[:-1]))
                    except Exception as e:
                        print(l)
                        print(e)

            if i > 6:
                t = line.split('\t')
                lat = t[0]
                if(lat[-1]=='S'):
                    latval = -1*float(lat[:-1])
                else:
                    latval = float(lat[:-1])

                # print(t)
                t = t[1:]
                tempvals = []
                sizes.append(len(t))
            i+=1
    return min(sizes), len(longitudes)

In [5]:
def process(filepath:str, min_of_max):
    '''
    Processes a ascii file with gridded data to obtain the required value in the form of [latitude, longitude, value.
        Args:
            filepath: str
        Returns:
            metadata: Dict
            values: List[List[float]]
    '''
    metadata = {}
    longitudes = []
    values = []
    sizes = []
    i = 0
    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if i == 0: metadata["varname"] = line.split(":")[1].strip()
            if i == 1: metadata["filename"] = line.split(":")[1].strip()
            if i == 2: pass
            if i == 3: metadata["badflag"] = line.split(":")[1].strip()
            if i == 4: metadata["subset"] = line.split(":")[1].strip()
            if i == 5: metadata["time"] = line.split(":")[1].strip()
            if i == 6:
                for idx, l in enumerate(line.split('\t')):
                    if idx == min_of_max:
                        break
                    try:
                        if l.strip()=='** line too long **':
                            continue
                        else:
                            longitudes.append(float(l.strip()[:-1]))
                    except Exception as e:
                        print(l)
                        print(e)
                        raise e

            if i > 6:
                t = line.split('\t')
                lat = t[0]
                if(lat[-1]=='S'):
                    latval = -1*float(lat[:-1])
                else:
                    latval = float(lat[:-1])

                t = t[1:]
                tempvals = []
                sizes.append(len(t))
                for idx, val in enumerate(t):
                    if idx == min_of_max:
                        break
                    if val.endswith('** line too long **'):
                        val = val.replace('** line too long **', '')
                        continue
                    try:
                        if(val==metadata["badflag"]):
                            tempvals = [longitudes[idx],latval,-1e34]
                        else:
                            tempvals = [longitudes[idx],latval,float(val)]
                        values.append(tempvals)
                    except Exception as e:
                        print()
                        print(idx)
                        print("Val", val)
                        print("t", t)
                        raise e
            i+=1
    return metadata, values

In [6]:
file_path = glob.glob('/content/drive/MyDrive/DV_Dataset/*.txt')
file_path.sort()

metadata_list = []
data_list = []
for file_path in file_path:
    metadata, data = process(filepath=file_path, min_of_max=-1)

    metadata_list.append(metadata)
    data_list.append(data)

date_lst = [
    '01-01-2023',
    '11-01-2023',
    '21-01-2023',
    '01-02-2023',
    '11-02-2023',
    '21-02-2023',
    '01-03-2023',
    '11-03-2023',
]

983


In [ ]:
for i, data in enumerate(data_list):
    longit, latit, values = zip(*data)

    latit = np.array(latit)
    longit = np.array(longit)
    values = np.array(values)
    mask = (values < -10)
    values[mask] = -1e10

    projection = ccrs.PlateCarree()

    fig = plt.figure(figsize=(10, 6))
    fig, ax = plt.subplots(subplot_kw={'projection': projection})

    start_value = -1
    stop_value = 32
    step_value = 0.05

    contour_values = np.arange(start_value, stop_value, step_value)

    contour = ax.tricontourf(longit, latit, values, transform=projection, levels=contour_values, cmap='tab10')

    ax.coastlines()

    cbar = plt.colorbar(contour, ax=ax, shrink=0.7)
    cbar.set_label('Value')

    ax.set_title(f'Sea Surface Temperature {date_lst[i]}')

    contour.colorbar.ax.set_ylabel("Temperature Values in Celsius")

    plt.savefig(f'{i+1}.png', dpi=600, bbox_inches='tight')
    plt.close()